****In this notebook I try to find the best features for xgboost. Anybody has anykind of advice or suggestion would appreciated****

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from reg_funcs import *

from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, train_test_split, StratifiedKFold
from xgboost import XGBRegressor
from copy import deepcopy
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from functools import partial
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

import eli5
from eli5.sklearn import PermutationImportance
from sklearn.metrics import make_scorer
from sklearn.inspection import permutation_importance
from sklearn.metrics import RocCurveDisplay, auc




from path import Path
import warnings 
warnings.filterwarnings('ignore') # supress warnings

In [2]:
# loading datasets
path = Path("/kaggle/input/playground-series-s3e11")

train = pd.read_csv(path / "train.csv")
test = pd.read_csv(path / "test.csv")
sub = pd.read_csv(path / "sample_submission.csv")


original_train = pd.read_csv("/kaggle/input/media-campaign-cost-prediction/train_dataset.csv")
original_test = pd.read_csv("/kaggle/input/media-campaign-cost-prediction/test_dataset.csv")
original = pd.concat([original_train, original_test])
original = original[train.drop('id', axis=1).columns]

original = original[~original['cost'].isnull()] # removing nulls
# removing duplicates in original
original = original[~original.drop("cost", axis=1).duplicated()]

In [ ]:
# converting ordinal categorical feature to integar
feats = ['unit_sales(in millions)', 'total_children', 'num_children_at_home', 'avg_cars_at home(approx).1', 'recyclable_package',
        'low_fat', 'units_per_case', 'store_sqft', 'coffee_bar', 'video_store', "salad_bar", "prepared_food", "florist"]

train[feats] = train[feats].astype('int')
test[feats] = test[feats].astype('int')
original[feats] = original[feats].astype('int')

****all the function I used****

In [ ]:
def evaluate_model(model_name, model, _X, _y, gbdt=True, original_data=None, use_original=False, n_splits=5, random_state_list=[0, 5, 10]):
    len_y = len(_y)
    len_states = len(random_state_list)

    oof_preds = np.zeros(len_y * len_states).reshape(len_states, len_y)
    models = []
    scores_train = []
    scalers = []
    for index, random_state in enumerate(random_state_list):
        print("#"*25)
        print("#"*15, f"traininng model {model_name} with seed {random_state}")
        print("#"*25)
        splitter = Splitter(n_splits=n_splits)
        splits = 0
        for X_train, X_val, y_train, y_val, train_idx, val_idx in splitter.split_data(_X, _y, random_state):

            
            if use_original: # we will only use original data for training not testing
                target = 'cost'
                X_train = pd.concat([X_train, original_data.drop(target, axis=1)]) 
                y_train = pd.concat([y_train, np.log(original_data[target])]) # only for  this playground series
                
#             all preprocessing will happen after this step

            if not gbdt:  
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_val = scaler.transform(X_val)
                scalers.append(deepcopy(scaler))
            
            _model = model()
            _model.fit(X_train, y_train)
            oof_preds[index, val_idx] = _model.predict(X_val).squeeze()
            models.append(deepcopy(_model))

            score_train = root_mean_squared_log_error(y_train, _model.predict(X_train))
            scores_train.append(score_train)

            score_valid_split = root_mean_squared_log_error(y_val, _model.predict(X_val).squeeze())

            print(f"seed {random_state} and split {splits} score {score_valid_split}")
            splits += 1

    oof_preds_mean = oof_preds.mean(axis=0)

    return models, oof_preds_mean, np.mean(scores_train), root_mean_squared_log_error(_y, oof_preds_mean), scalers
   
    
    
def predict_test(models, X_test, scalers=None, gbdt=True, n_splits=5, n_repeats=3):
    test_preds = np.zeros(n_splits * n_repeats * len(X_test)).reshape(n_splits * n_repeats, len(X_test))
    X_test_ = X_test.copy()
    for index, model in enumerate(models):
        if not gbdt:
            X_test_ = scalers[index].transform(X_test_) # normally you would not do ensembling for linear models 
        preds = model.predict(X_test_)                 # if we have a lot of observations we can do that if we want
        test_preds[index, range(len(preds))] = preds
        
    return test_preds.mean(axis=0)

    
#  plot correaltion        
def plot_corr(df, features, target, sort=False, method='pearson', figsize=(13, 8), use_mask=True, mask_type="triu", **kwargs):
    """Plot correlation given dataset and features names"""
    plt.figure(figsize=figsize) # sets figure size
    corr = df[features].corr(method=method) # calculates correlation based on the features and method provied
    target_values = corr[target]
    features.remove(target)
    corr = corr[features]
    
    corr = pd.concat([corr, target_values], axis=1)
    
    if sort: # if sort is true then sort the correlation matrix
        corr = corr.sort_values(by=target)
    if use_mask: # if uses_mask = True
        if mask_type == 'triu': # sets mask type to lower trigonal
            mask = np.triu(np.ones(corr.shape)) 
        else:   # sets mask type to upper trigonal
            mask = np.tril(np.ones(corr.shape))
        sns.heatmap(corr, annot=True, mask=mask, **kwargs)
        
    else: # if uses mask is not true
         sns.heatmap(corr, annot=True, **kwargs)       

    plt.title('Correlation between features')
  



def plot_importance(models, X_test, title=""):
#     taken from https://www.kaggle.com/code/shoabahamed/ps3e9-eda-and-gbdt-catboost-median-duplicatedata/edit
    """Plots features importance given models and train set"""
    features = X_test.columns.tolist()
    feature_importance = pd.DataFrame()
    for model in models:
        _df = pd.DataFrame()
        _df['importance'] = model.feature_importances_
        _df["features"] = pd.Series(features)
        _df = _df.sort_values(by='importance', ascending=False)
        feature_importance = pd.concat([feature_importance, _df])
        
                
    feature_importance = feature_importance.sort_values('importance', ascending=False)
    plt.figure(figsize=(16, 10))
    ax = sns.barplot(x='importance', y='features', data=feature_importance, color='skyblue', errorbar='sd')
    
    for i in ax.containers:
        ax.bar_label(i,)
    
   
    plt.xlabel('Importance', fontsize=14)
    plt.ylabel('Feature', fontsize=14)
    plt.title(f"{title} Feature Importances", fontsize=18)
    plt.grid(True, axis='x')
    plt.show()
    
    return feature_importance



def root_mean_squared_log_error(y_true_log, y_pred_log):
    return mean_squared_error(y_true_log, y_pred_log, squared=False)


def plot_residuals(df, y_true, y_preds, features, target, **kwargs):
    """Takes the trained dataset, y_true and y_preds. You also have to provide the feature names used in training and target feature\
    name"""
    cols = 3
    rows = int(np.ceil(len(features) / 3))
    
    fig, ax = plt.subplots(rows, cols, figsize=(15, rows*4))
    axes =  ax.flatten()
    
    if not isinstance(y_true, np.ndarray):
        y_true = y_true.values
    if not isinstance(y_preds, np.ndarray):
        y_preds = y_preds.values
        
    y_res = y_true - y_preds
    
    for index, feat in enumerate(features):      
        ax = sns.scatterplot(data=df, x=feat, y=y_res, ax=axes[index], **kwargs)

        ax.axhline(0, color='red', ls='--')
        ax.set_ylabel('y residuals')
        if feat == target:
            ax.set_xlabel(f"target - {target} true values")
            continue
        ax.set_xlabel(feat)

    plt.suptitle(f"Residual plots for features", size=14)
    plt.tight_layout()




class Splitter:
    """A splitter class which splits the X, y using the split_data function with a random state provided. It yeilds \
    X_train, X_val, y_train, y_val, train_idx, val_idx in the end.\
    code from  https://www.kaggle.com/code/tetsutani/ps3e9-eda-and-gbdt-catboost-median-duplicatedata wit little bit of modification """

    def __init__(self, test_size=0.2, kfold=True, n_splits=5):
        self.test_size = test_size # set test size
        self.kfold = kfold  # wheter to just split the data in two or use kfold
        self.n_splits= n_splits # set 
        
    def split_data(self, X, y, random_state):
        if self.kfold:
            kf = KFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
            for train_idx, val_idx in kf.split(X, y):
                X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
                y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
                yield X_train, X_val, y_train, y_val, train_idx, val_idx
        else:
            X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=self.test_size, random_state=random_state)
            yield X_train, X_val, y_train, y_val

            
# plot adversarial_validations scores for each splits
def adversarial_validation(train_df, test_df, classifier, n_splits=5, title='Roc curves', figsize=(6, 6)):
    """This is a function which checks if distribution between two datasets are same given the same columns. Then plots roc curve\
for each split. A score close to 0.5 means very similar(Do not provide the target feature)"""
    
    train_df['target'] = 1  
    test_df['target'] = 0
    
    if len(train_df) > len(test_df):
        size = len(test_df)
    else:
        size = len(train_df)
    
    train_df = train_df.sample(size)
    test_df = test_df.sample(size)
    
    df = pd.concat([train_df, test_df]).sample(frac=1)
    
    X = df.drop('target', axis=1)
    y = df['target']
    
    cv = StratifiedKFold(n_splits=n_splits)

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    fig, ax = plt.subplots(figsize=figsize)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        classifier.fit(X.iloc[train_idx], y.iloc[train_idx])
        viz = RocCurveDisplay.from_estimator(
            classifier,
            X.iloc[val_idx],
            y.iloc[val_idx],
            name=f"ROC fold {fold}",
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)
        
    ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"{title}",
    )
    ax.axis("square")
    ax.legend(loc="lower right")
    plt.show()


# plot pca
def plot_pca(df):
    """Takes a dataset and do pricinpal component analysis and plots the results"""
    #     The code below is mostly taken from kaggle learn course Principal Component Analysis. Link is below
    # https://www.kaggle.com/code/ryanholbrook/principal-component-analysis#Example---1985-Automobiles
    _X = df.copy()
    features = _X.columns.tolist()

    # Standardize
    X_scaled = (_X - _X.mean(axis=0)) / _X.std(axis=0)
    
    pca = PCA()
    X_pca = pca.fit_transform(X_scaled)

    # Convert to dataframe
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(X_pca, columns=component_names)

    loadings = pd.DataFrame(
        pca.components_.T,  # transpose the matrix of loadings
        columns=component_names,  # so the columns are the principal components
        index=_X.columns,  # and the rows are the original features
    )
    
    display(loadings)
    plot_variance(pca)
    
    return X_pca



# helper function of plot_pca
def plot_variance(pca, width=8, dpi=100):
    """"plot pca vairance given the pca object provided where the dataset is already passed"""
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=12, dpi=100)
    return axs
    

In [ ]:
# can try to see if rounding continous features increases the score
np.round(train['store_sales(in millions)']).nunique()

****Correlation****

In [ ]:
features = train.drop('id', axis=1).columns.tolist()
plot_corr(train.drop('id', axis=1), features, target='cost', sort=True, cmap="Greens", fmt='.3f', mask_type='tril')

****Baseline Model****

Any experiment we do has must have better than this model to be accepeted

In [ ]:
X = train.drop(['id', 'cost', 'salad_bar'], axis=1)
y = np.log(train['cost'])

features = X.columns.tolist()
xgb = partial(XGBRegressor, tree_method='gpu_hist', random_state=0)

In [ ]:
_models, _oof_preds, mean_train_score, mean_valid_score, _scalers = evaluate_model("XGBRegressor_default", xgb, X, y, gbdt=True, 
                                                                            n_splits=10, random_state_list=[0, 5, 10]) 

In [ ]:
print(f"Default r2 score: {r2_score(y, _oof_preds)}")

print(f"default validation rmsle score: ", mean_valid_score)

****Feature Importance in baseline****

In [ ]:
feature_importances = plot_importance(_models, test.drop(['id', 'salad_bar'], axis=1), title='Splits')

It seems that order of most important features imporatance are constant. Lets try to train the model on whole dataset and see how the plot looks. Although video stores shows some variability with store_sqft

In [ ]:
_model1 = XGBRegressor(random_state=0, tree_method='gpu_hist', silent=True)
_model1.fit(X, y)

feature_importances_all = plot_importance([_model1], test.drop(['id', 'salad_bar'], axis=1), title='All data')

Other than store sqft and video store the importance order are mostly similar to ensemble model

## Residual

In [ ]:
# in ensemble model
y_train_preds = predict_test(_models, X, gbdt=True, n_splits=10, n_repeats=3)
print(f"The r2 scoer is {r2_score(y, y_train_preds)} for training")
print(f"The r2 scoer is {r2_score(y, _oof_preds)} for validataion")

In [ ]:
sns.distplot(y)
sns.distplot(_oof_preds)
plt.legend(['train true values', 'train_predictions'])
plt.title("Distribution of true and predicted");

Our predictions dispite scoring well in the leaderboard they actually does not follow the same distribution as training . Lets dig a little depper and see if we can find out how much our model was not able to learn from residuals

In [ ]:
plot_residuals(train, y, _oof_preds, features+['cost'], 'cost')

## Permutaition Importance

You can learn about Permutation Importance from [here](https://www.kaggle.com/code/dansbecker/permutation-importance). I also liked how Permutation Importance was displayed in this notebook by https://www.kaggle.com/code/janmpia/what-you-need-to-know-about-this-competition by <b>J€ANMPIA</b>

In [ ]:
scorer = make_scorer(root_mean_squared_log_error, greater_is_better=False)

In [ ]:
# for different splitting
splitter = Splitter(kfold=False)
X_train, X_val, y_train, y_val = next(splitter.split_data(X, y, random_state=0))
feats = X_train.columns.tolist()

_model = partial(XGBRegressor, random_state=0, tree_method='hist')()
_model.fit(X_train, y_train)

perm = PermutationImportance(_model, scoring=scorer).fit(X_val, y_val)
eli5.show_weights(perm, feature_names=feats)

In [ ]:
splitter = Splitter(kfold=False)
X_train, X_val, y_train, y_val = next(splitter.split_data(X, y, random_state=10)) 
                                                                                 
_model = partial(XGBRegressor, random_state=0, tree_method='hist')()
_model.fit(X_train, y_train)

perm = PermutationImportance(_model, random_state=0, scoring=scorer).fit(X_val, y_val)
eli5.show_weights(perm, feature_names=X.columns.tolist())

Here we can see that store sqft is the most important feature which is incosistent with feature importance plot above. Although florist is the second most important one. Another thing to notice is that low_fat and recyclable_package shows very little negative values which mean that it was probably not an important feature or it just happend by chance. Anyway we can try removing these features than train our model to submit.(I have tried that and both leaderboard and local valid cv score has increased very little

Problems we need to deal with:
* First of all model is not learning enough pattern from the features. It could be because of two much noise or too few features. It can also happen because of bad hyperparamters.
     
How to solve it:
* Try to add more data and see if works(adding original)
* Then we will do PCA and see if it reduces our noise in the features
* Second we will try to come up with new features from existing ones or remove features during training: 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case' were unimportant in correlation, feature_importance, permutation importance. So we will remove them

## Adding original

In [ ]:
train_all = pd.concat([train.drop('id', axis=1), original])
train_all.head(2)

In [ ]:
train_all.drop(['cost'], axis=1).duplicated(keep=False).sum()

for the same observation 13864 instances where cost is different which 3% of the whole dataset. we will only keep the first ones
which reperesent the train dataset 

In [ ]:
train_all = train_all.loc[~train_all.drop(['cost'], axis=1).duplicated(keep='first')] # only taking the first occurences

lets see if there are any matches between train all and test

In [ ]:
pd.concat([train_all.drop(columns=['cost']), test.drop('id', axis=1)]).duplicated(keep=False).sum()

There are few matches between train_all and test due to including original. Lets check if distribution is similar in train cost and original cost

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
adversarial_validation(original[['cost']], train[['cost']], LGBMClassifier())

distribution of orginal cost and train cost is the same. We can see there are not much difference in distribtuion between train data original data

In [ ]:
adversarial_validation(original.drop('cost', axis=1), train.drop(['id', 'cost'], axis=1), LGBMClassifier())

Okay lets train the model and see how much our score increases or decreases by including original data

In [ ]:
org = original.copy()
train_temp = train.copy()

org['generated'] = False
train_temp['generated'] = True

data = pd.concat([train_temp.drop('id', axis=1), org])
data = data[~data.drop(['cost', 'generated'], axis=1).duplicated(keep='first')]

data.head()

In [ ]:
X_all = train.drop(['id', 'cost', 'salad_bar'], axis=1)
y_all = np.log(train['cost'])
features_all = X_all.columns.tolist()
add_data = data.loc[data['generated'] == False]
add_data = add_data[features_all + ['cost']]

In [ ]:
_models_all, _oof_preds_all, mean_train_score_all, mean_valid_score_all, _scalers_all = evaluate_model("XGBRegressor", xgb, X_all, y_all, 
                                                                            gbdt=True, use_original=True, original_data=add_data,
                                                                            n_splits=10, random_state_list=[0, 5, 10]) 

In [ ]:
print(f"valid score default: {mean_valid_score}")
print(f"valid score with all data addded: {mean_valid_score_all}")

In [ ]:
test_preds_all = predict_test(_models_all, test[features_all], scalers=_scalers_all, gbdt=True, n_splits=10, n_repeats=3)

In [ ]:
def submission_csv(predictions, target='cost'):
    df = pd.DataFrame()
    df['id'] = test['id']
    df[target] = np.exp(predictions) # exp only for this rmsle
    
    return df

df_all_data = submission_csv(test_preds_all, target='cost')

In [ ]:
# df_all_data.to_csv("xgb_all_data_correct.csv", index=False)

The score has very little. But I am not sure even though the score if it is worth it to include original. check this thread https://www.kaggle.com/competitions/playground-series-s3e11/discussion/397431 by J€ANMPIA.

But we will include original as it has increased our score in both local cv and leaderboard although not much

## Removing Less important features

Removing noisy or unimportant features may improve our r2score and rmsle. 

In [ ]:
X_all_rm = train.drop(columns=['id', 'cost', 'salad_bar', 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case'])
y_all_rm = np.log(train['cost'])
features_all_rm = X_all_rm.columns.tolist()

_models_all_rm, _oof_preds_all_rm, mean_train_score_all_rm, mean_valid_score_all_rm, _scalers_all_rm = evaluate_model("XGBRegressor", 
                                    xgb, X_all_rm, y_all_rm, gbdt=True, n_splits=10, random_state_list=[0, 5, 10]) 

In [ ]:
print(f"Default r2 score: {r2_score(y, _oof_preds)}")
print(f"r2 score after removing 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case': {r2_score(y_all_rm, _oof_preds_all_rm)}")

print()

print(f"default validation rmsle score: ", mean_valid_score)
print(f"validation rmslse after removing 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case': ",mean_valid_score_all_rm)

In [ ]:
test_preds_all_rm = predict_test(_models_all_rm, test[features_all_rm], gbdt=True, n_splits=10, n_repeats=3)

In [ ]:
xgb_model_all_rm = submission_csv(test_preds_all_rm)
xgb_model_all_rm.head()

In [ ]:
# xgb_model_all_rm.to_csv("xgb_model_all_rm_correct.csv", index=False)

## PCA

In [ ]:
X_pca = plot_pca(X)

In [ ]:
X_pca.head()

In [ ]:
_models_pca, _oof_preds_pca, mean_train_score_pca, mean_valid_score_pca, _scalers_pca = evaluate_model("XGBRegressor_default", 
                                                                xgb, X_pca, y, gbdt=True, n_splits=10, random_state_list=[0, 5, 10]) 

In [ ]:
print(f"pca trained local valid score {mean_valid_score_pca}")
sns.displot(_oof_preds_pca);

Well pca is not helping. Lets try to see if creating new features helps. We will do this using 
- feature importance plot
- permutation imporatnce
- partial importance 
- correlation (which will not apply in this dataset)
- shap

We have done feature importance, correlation and permutation importance before

In [ ]:
# from pca, feature_importance and permutation computations and after some trial and error
train_temp = train.copy(deep=True)

train_temp['extra_attraction'] = train_temp['florist'] + train_temp['video_store'] + train_temp['prepared_food'] + train_temp['coffee_bar']
train_temp['children'] = train_temp['total_children'] * train_temp['num_children_at_home']

plot_corr(train_temp, train_temp.columns.tolist(), target='cost', cmap='Greens', sort=True, mask_type='tril', use_mask=False, fmt='.2f')

In [ ]:
# display(train_temp.pivot_table(values='id', columns='num_children_at_home', index='total_children', aggfunc='count'))
# filt = train_temp['num_children_at_home'] > train_temp['total_children']
# train_temp.loc[filt, 'num_children_at_home'] = train_temp.loc[filt, 'total_children']
# train_temp.pivot_table(values='id', columns='num_children_at_home', index='total_children', aggfunc='count')

In [ ]:
# removing more features and adding feature engineered features
X_all_rms = train.drop(columns=['id', 'cost', 'salad_bar', 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case', 'store_sales(in millions)', 'unit_sales(in millions)'])
y_all_rms = np.log(train['cost'])
features_all_rms = X_all_rms.columns.tolist()
add_data = data.loc[data['generated'] == False]
add_data = add_data[features_all_rms + ['cost']]

# creating features
X_all_rms['extra_attraction'] = X_all_rms['florist'] + X_all_rms['video_store'] + X_all_rms['prepared_food'] + X_all_rm['coffee_bar']
X_all_rms['children'] = X_all_rms['total_children'] * X_all_rms['num_children_at_home']

add_data['extra_attraction'] = add_data['florist'] + add_data['video_store'] + add_data['prepared_food'] + add_data['coffee_bar']
add_data['children'] = add_data['total_children'] * add_data['num_children_at_home']

features_all_rms = X_all_rms.columns.tolist()

_models_all_rms, _oof_preds_all_rms, mean_train_score_all_rms, mean_valid_score_all_rms, _scalers_all_rms = evaluate_model("XGBRegressor_default", xgb, 
                                                                X_all_rms, y_all_rms , use_original=True, original_data=add_data, gbdt=True, n_splits=10, random_state_list=[0, 5, 10]) 

In [ ]:
print(f"Default r2 score: {r2_score(y, _oof_preds)}")
print(f"r2 score after removing 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case': {r2_score(y_all_rms, _oof_preds_all_rms)}")

print()

print(f"default validation rmsle score: ", mean_valid_score)
print(f"validation rmslse after removing 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case': ",mean_valid_score_all_rms)

In [ ]:
test_temp = test.copy()
test_temp['extra_attraction'] = test_temp['florist'] + test_temp['video_store'] + test_temp['prepared_food'] + test_temp['coffee_bar']
test_temp['children'] = test_temp['total_children'] * test_temp['num_children_at_home']

test_preds_all_rms = predict_test(_models_all_rms, test_temp[features_all_rms], gbdt=True, n_splits=10, n_repeats=3)

In [ ]:
df_all_rms = submission_csv(test_preds_all_rms)
df_all_rms.head()

In [ ]:
# df_all_rms.to_csv("xgb_all_rms.csv", index=False)

****partial dependence plot****

Learn more about parital plot from [here](https://www.kaggle.com/code/dansbecker/partial-plots/tutorial)

In [ ]:
X_par = train.drop(columns=['id', 'cost', 'salad_bar', 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case'])
y_par = np.log(train['cost'])
features_par = X_par.columns.tolist()
add_data = data.loc[data['generated'] == False]
add_data = add_data[features_par+ ['cost']]




splitter = Splitter(kfold=False)
X_train, X_val, y_train, y_val = next(splitter.split_data(X_par, y_par, random_state=5))
X_train = pd.concat([X_train, add_data.drop('cost', axis=1)])
y_train = pd.concat([y_train, np.log(add_data['cost'])])

_model_par = xgb()
_model_par.fit(X_train, y_train)

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

feat_name = 'florist'

PartialDependenceDisplay.from_estimator(_model_par, X_val, features=[feat_name])
plt.show()

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

feat_name = 'florist'

fig, ax = plt.subplots(1, 1, figsize=(6, 5))
ax = PartialDependenceDisplay.from_estimator(_model_par, X_val, features=[feat_name], kind='both', ax=ax)

plt.show()

In [ ]:
sns.boxplot(data=train, x='florist', y='cost')

we can that in our model there is  a downward trend in although the trend is different each time ther and there are some times where the trend is upwards

In [ ]:
feat_name = ('video_store', 'florist')


fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax = PartialDependenceDisplay.from_estimator(_model_par, X_val, features=[feat_name], kind='average', ax=ax)

plt.show()

In [ ]:
feat_name = ('coffee_bar', 'florist')


fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax = PartialDependenceDisplay.from_estimator(_model_par, X_val, features=[feat_name], kind='average', ax=ax)

plt.show()

In [ ]:
feat_name = ('prepared_food', 'florist')


fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax = PartialDependenceDisplay.from_estimator(_model_par, X_val, features=[feat_name], kind='average', ax=ax)

plt.show()

All of these extra features shows a downward trend when they are together. We have captured that before with adding them together

In [ ]:
feat_name = ('total_children', 'num_children_at_home')


fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax = PartialDependenceDisplay.from_estimator(_model_par, X_val, features=[feat_name], kind='average', ax=ax)

plt.show()

In [ ]:
feat_name = ('avg_cars_at home(approx).1', 'total_children')


fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax = PartialDependenceDisplay.from_estimator(_model_par, X_val, features=[feat_name], kind='average', ax=ax)

plt.show()

avg_car_at_home shows similar pattern with total children as no_children_at_home. So multiplying all of them together may increase our score

In [ ]:
X_all_rms = train.drop(columns=['id', 'cost', 'salad_bar', 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case', 'store_sales(in millions)', 'unit_sales(in millions)'])
y_all_rms = np.log(train['cost'])
features_all_rms = X_all_rms.columns.tolist()
add_data = data.loc[data['generated'] == False]
add_data = add_data[features_all_rms + ['cost']]

# creating features
X_all_rms['extra_attraction'] = X_all_rms['florist'] + X_all_rms['video_store'] + X_all_rms['prepared_food'] + X_all_rms['coffee_bar']
X_all_rms['children'] = X_all_rms['total_children'] * X_all_rms['num_children_at_home']
X_all_rms['children*avg_cars_at home(approx).1'] = X_all_rms['total_children'] * X_all_rms['num_children_at_home'] * X_all_rms['avg_cars_at home(approx).1']

add_data['extra_attraction'] = add_data['florist'] + add_data['video_store'] + add_data['prepared_food'] + add_data['coffee_bar']
add_data['children'] = add_data['total_children'] * add_data['num_children_at_home']
add_data['children*avg_cars_at home(approx).1'] = add_data['total_children'] * add_data['num_children_at_home']* add_data['avg_cars_at home(approx).1']

features_all_rms = X_all_rms.columns.tolist()

_models_all_rms, _oof_preds_all_rms, mean_train_score_all_rms, mean_valid_score_all_rms, _scalers_all_rms = evaluate_model("XGBRegressor_default", xgb, 
                                                                X_all_rms, y_all_rms , use_original=True, original_data=add_data, gbdt=True, n_splits=10, random_state_list=[0, 5, 10]) 

In [ ]:
print(f"r2 score after removing 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case': {r2_score(y_all_rms, _oof_preds_all_rms)}")
print(f"validation rmslse after removing 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case': ",mean_valid_score_all_rms)

In [ ]:
test_temp = test.copy()
test_temp['extra_attraction'] = test_temp['florist'] + test_temp['video_store'] + test_temp['prepared_food'] + test_temp['coffee_bar']
test_temp['children'] = test_temp['total_children'] * test_temp['num_children_at_home']
test_temp['children*avg_cars_at home(approx).1'] = test_temp['children'] * test_temp['avg_cars_at home(approx).1']


test_preds_all_rms = predict_test(_models_all_rms, test_temp[features_all_rms], gbdt=True, n_splits=10, n_repeats=3)

In [ ]:
df_all_rms = submission_csv(test_preds_all_rms)
df_all_rms.to_csv("xgb_all_rms_improved_correct.csv", index=False)

The score has increased. But store_sqft is very important feature but the model was not able to capture it. Let 

****SHAP(SHapley Additive exPlanations)****

Learn about shap from [here](https://www.kaggle.com/code/dansbecker/shap-values) 

In [ ]:
X_shap = train.drop(columns=['id', 'cost', 'salad_bar', 'gross_weight', 'low_fat', 
                            'recyclable_package', 'units_per_case', 'store_sales(in millions)', 'unit_sales(in millions)'])
y_shap = np.log(train['cost'])
features_par = X_shap.columns.tolist()
add_data = data.loc[data['generated'] == False]
add_data = add_data[features_par+ ['cost']]

X_shap['extra_attraction'] = X_shap['florist'] + X_shap['video_store'] + X_shap['prepared_food'] + X_shap['coffee_bar']
X_shap['children'] = X_shap['total_children'] * X_shap['num_children_at_home']
X_shap['children*avg_cars_at home(approx).1'] = X_shap['children'] * X_shap['avg_cars_at home(approx).1']

add_data['extra_attraction'] = add_data['florist'] + add_data['video_store'] + add_data['prepared_food'] + add_data['coffee_bar']
add_data['children'] = add_data['total_children'] * add_data['num_children_at_home']
add_data['children*avg_cars_at home(approx).1'] = add_data['children'] * add_data['avg_cars_at home(approx).1']



splitter = Splitter(kfold=False)
X_train, X_val, y_train, y_val = next(splitter.split_data(X_shap, y_shap, random_state=5))
X_train = pd.concat([X_train, add_data.drop('cost', axis=1)])
y_train = pd.concat([y_train, np.log(add_data['cost'])])

_model_shap = xgb()
_model_shap.fit(X_train, y_train)

In [ ]:
import shap

explainer = shap.TreeExplainer(_model_shap)

# Calculate Shap values
shap_values = explainer.shap_values(X_val)

In [ ]:
i = 7
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[i], X_val.iloc[i] ,feature_names=X_val.columns.to_list())

In [ ]:
i = 5
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[i], X_val.iloc[i] ,feature_names=X_val.columns.to_list())

In [ ]:
# shap.force_plot(explainer.expected_value, shap_values[0:6,:], X_val.iloc[0:6,:], plot_cmap="DrDb",
#                 feature_names=X_val.columns.to_list())

In [ ]:
shap.summary_plot(shap_values, X_val, feature_names=X_val.columns.tolist())

In [ ]:
shap.dependence_plot('store_sqft', shap_values, X_val, interaction_index='avg_cars_at home(approx).1')

In [ ]:
shap.dependence_plot('store_sqft', shap_values, X_val)

In [ ]:
# sns.scatterplot(data=train.iloc[:1000], y='cost', x='id', hue='store_sqft', size='store_sqft')

need to change evalue_model and predict model a little

In [ ]:
# need to change evaluation method a little
def evaluate_model(model_name, model, _X, _y, gbdt=True, original_data=None, use_original=False, n_splits=5, random_state_list=[0, 5, 10]):
    len_y = len(_y)
    len_states = len(random_state_list)

    oof_preds = np.zeros(len_y * len_states).reshape(len_states, len_y)
    models = []
    scores_train = []
    scalers = []
    encoders = []
    for index, random_state in enumerate(random_state_list):
        print("#"*25)
        print("#"*15, f"traininng model {model_name} with seed {random_state}")
        print("#"*25)
        splitter = Splitter(n_splits=n_splits)
        splits = 0
        for X_train, X_val, y_train, y_val, train_idx, val_idx in splitter.split_data(_X, _y, random_state):
    
            
            if use_original: # we will only use original data for training not testing
                target = 'cost'
                X_train = pd.concat([X_train, original_data.drop(target, axis=1)]) 
                y_train = pd.concat([y_train, np.log(original_data[target])]) # only for 
                
#             all preprocessing will happen after this step

            if not gbdt:  
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_val = scaler.transform(X_val)
                scalers.append(deepcopy(scaler))
              
            
            encoder = MEstimateEncoder(cols='store_sqft_encode', m=10)
#             encoder = TargetEncoder(cols='store_sqft_encode')
            X_train = encoder.fit_transform(X_train, y_train)
            X_val = encoder.transform(X_val)
            encoders.append(deepcopy(encoder))


            _model = model()
            _model.fit(X_train, y_train)
            oof_preds[index, val_idx] = _model.predict(X_val).squeeze()
            models.append(deepcopy(_model))

            score_train = root_mean_squared_log_error(y_train, _model.predict(X_train))
            scores_train.append(score_train)

            score_valid_split = root_mean_squared_log_error(y_val, _model.predict(X_val).squeeze())

            print(f"seed {random_state} and split {splits} score {score_valid_split}")
            splits += 1
            

        
        X_train.head()

    oof_preds_mean = oof_preds.mean(axis=0)

    return models, oof_preds_mean, np.mean(scores_train), root_mean_squared_log_error(_y, oof_preds_mean), scalers, encoders


def predict_test(models, X_test, scalers=None, encoders=None, gbdt=True, n_splits=5, n_repeats=3):
    test_preds = np.zeros(n_splits * n_repeats * len(X_test)).reshape(n_splits * n_repeats, len(X_test))
    for index, model in enumerate(models):
        X_test_ = X_test.copy()
        if not gbdt:
            X_test_ = scalers[index].transform(X_test_) # normally you would not do ensembling for linear models 
        
        if encoders:
#             print("Sdjf")
            X_test_ = encoders[index].transform(X_test_)
#             display(X_test_.head(3))
        preds = model.predict(X_test_)                 # if we have a lot of observations we can do that if we want
        test_preds[index, range(len(preds))] = preds
        
    return test_preds.mean(axis=0)


In [ ]:
# success
X_all_rms = train.drop(columns=['id', 'cost', 'salad_bar', 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case', 'store_sales(in millions)', 'unit_sales(in millions)'])
y_all_rms = np.log(train['cost'])
features_all_rms = X_all_rms.columns.tolist()
add_data = data.loc[data['generated'] == False]
add_data = add_data[features_all_rms + ['cost']]

# creating features
X_all_rms['extra_attraction'] = X_all_rms['florist'] + X_all_rms['video_store'] + X_all_rms['prepared_food'] + X_all_rms['coffee_bar']
X_all_rms['children'] = X_all_rms['total_children'] * X_all_rms['num_children_at_home']
X_all_rms['children*avg_cars_at home(approx).1'] = X_all_rms['total_children'] * X_all_rms['num_children_at_home'] * X_all_rms['avg_cars_at home(approx).1']
X_all_rms['store_sqft_encode'] = X_all_rms['store_sqft'].copy()

add_data['extra_attraction'] = add_data['florist'] + add_data['video_store'] + add_data['prepared_food'] + add_data['coffee_bar']
add_data['children'] = add_data['total_children'] * add_data['num_children_at_home']
add_data['children*avg_cars_at home(approx).1'] = add_data['total_children'] * add_data['num_children_at_home']* add_data['avg_cars_at home(approx).1']
add_data['store_sqft_encode'] = add_data['store_sqft'].copy()
features_all_rms = X_all_rms.columns.tolist()

_models_all_rms, _oof_preds_all_rms, mean_train_score_all_rms, mean_valid_score_all_rms, _scalers_all_rms, encoders_all_rms = evaluate_model("XGBRegressor_default", xgb, 
                                                                X_all_rms, y_all_rms , use_original=True, original_data=add_data, gbdt=True, n_splits=10, random_state_list=[0, 5, 10]) 

In [ ]:
print(f"Best score till now: ", mean_valid_score_all_rms)

In [ ]:
test_temp = test.copy()
test_temp['extra_attraction'] = test_temp['florist'] + test_temp['video_store'] + test_temp['prepared_food'] + test_temp['coffee_bar']
test_temp['children'] = test_temp['total_children'] * test_temp['num_children_at_home']
test_temp['children*avg_cars_at home(approx).1'] = test_temp['children'] * test_temp['avg_cars_at home(approx).1']
test_temp['store_sqft_encode'] = test_temp['store_sqft'].copy()

test_preds_all_rms = predict_test(_models_all_rms, test_temp[features_all_rms], encoders=encoders_all_rms, gbdt=True, n_splits=10, n_repeats=3)

In [ ]:
df_encode_store_sqft = submission_csv(test_preds_all_rms)

In [ ]:
df_encode_store_sqft.to_csv("xgb_all_rms_encode.csv", index=False)

Lest combine some other models and build an ensemble model

In [ ]:
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
# xgb = partial(XGBRegressor, tree_method='gpu_hist', random_state=0)
# cat = partial(CatBoostRegressor, random_state=0, verbose=False, task_type="GPU")
# lgbm = partial(LGBMRegressor, random_state=0, device='gpu')

In [ ]:
# # success
# X_all_rms = train.drop(columns=['id', 'cost', 'salad_bar', 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case', 'store_sales(in millions)', 'unit_sales(in millions)'])
# y_all_rms = np.log(train['cost'])
# features_all_rms = X_all_rms.columns.tolist()
# add_data = data.loc[data['generated'] == False]
# add_data = add_data[features_all_rms + ['cost']]

# # creating features
# X_all_rms['extra_attraction'] = X_all_rms['florist'] + X_all_rms['video_store'] + X_all_rms['prepared_food'] + X_all_rms['coffee_bar']
# X_all_rms['children'] = X_all_rms['total_children'] * X_all_rms['num_children_at_home']
# X_all_rms['children*avg_cars_at home(approx).1'] = X_all_rms['total_children'] * X_all_rms['num_children_at_home'] * X_all_rms['avg_cars_at home(approx).1']
# X_all_rms['store_sqft_encode'] = X_all_rms['store_sqft'].copy()

# add_data['extra_attraction'] = add_data['florist'] + add_data['video_store'] + add_data['prepared_food'] + add_data['coffee_bar']
# add_data['children'] = add_data['total_children'] * add_data['num_children_at_home']
# add_data['children*avg_cars_at home(approx).1'] = add_data['total_children'] * add_data['num_children_at_home']* add_data['avg_cars_at home(approx).1']
# add_data['store_sqft_encode'] = add_data['store_sqft'].copy()
# features_all_rms = X_all_rms.columns.tolist()

# # _models_all_rms_xgb, _oof_preds_all_rms_xgb, mean_train_score_all_rms_xgb, mean_valid_score_all_rms_xgb, _scalers_all_rms_xgb, encoders_all_rms_xgb = evaluate_model("XGBRegressor_default", xgb, 
# #                                                                 X_all_rms, y_all_rms , use_original=True, original_data=add_data, gbdt=True, n_splits=10, random_state_list=[0, 5, 10]) 

# _models_all_rms_cat, _oof_preds_all_rms_cat, mean_train_score_all_rms_cat, mean_valid_score_all_rms_cat, _scalers_all_rms_cat, encoders_all_rms_cat = evaluate_model("catboost_default", cat, 
#                                                                 X_all_rms, y_all_rms , use_original=True, original_data=add_data, gbdt=True, n_splits=10, random_state_list=[0, 5, 10]) 


In [ ]:
# mean_valid_score_all_rms_cat

In [ ]:
# root_mean_squared_log_error(y_all_rms, (_oof_preds_all_rms_lgbm + _oof_preds_all_rms)/2)

In [ ]:
# # success
# X_all_rms = train.drop(columns=['id', 'cost', 'salad_bar', 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case', 'store_sales(in millions)', 'unit_sales(in millions)'])
# y_all_rms = np.log(train['cost'])
# features_all_rms = X_all_rms.columns.tolist()
# add_data = data.loc[data['generated'] == False]
# add_data = add_data[features_all_rms + ['cost']]

# # creating features
# X_all_rms['extra_attraction'] = X_all_rms['florist'] + X_all_rms['video_store'] + X_all_rms['prepared_food'] + X_all_rms['coffee_bar']
# X_all_rms['children'] = X_all_rms['total_children'] * X_all_rms['num_children_at_home']
# X_all_rms['children*avg_cars_at home(approx).1'] = X_all_rms['total_children'] * X_all_rms['num_children_at_home'] * X_all_rms['avg_cars_at home(approx).1']
# X_all_rms['store_sqft_encode'] = X_all_rms['store_sqft'].copy()

# add_data['extra_attraction'] = add_data['florist'] + add_data['video_store'] + add_data['prepared_food'] + add_data['coffee_bar']
# add_data['children'] = add_data['total_children'] * add_data['num_children_at_home']
# add_data['children*avg_cars_at home(approx).1'] = add_data['total_children'] * add_data['num_children_at_home']* add_data['avg_cars_at home(approx).1']
# add_data['store_sqft_encode'] = add_data['store_sqft'].copy()
# features_all_rms = X_all_rms.columns.tolist()

# _models_all_rms, _oof_preds_all_rms, mean_train_score_all_rms, mean_valid_score_all_rms, _scalers_all_rms, encoders_all_rms = evaluate_model("XGBRegressor_default", xgb, 
#                                                                 X_all_rms, y_all_rms , use_original=True, original_data=add_data, gbdt=True, n_splits=10, random_state_list=[0, 5, 10]) 

In [ ]:
# # success
# X_all_rms = train.drop(columns=['id', 'cost', 'salad_bar', 'gross_weight', 'low_fat', 'recyclable_package', 'units_per_case', 'store_sales(in millions)', 'unit_sales(in millions)'])
# y_all_rms = np.log(train['cost'])
# features_all_rms = X_all_rms.columns.tolist()
# add_data = data.loc[data['generated'] == False]
# add_data = add_data[features_all_rms + ['cost']]

# # creating features
# X_all_rms['extra_attraction'] = X_all_rms['florist'] + X_all_rms['video_store'] + X_all_rms['prepared_food'] + X_all_rms['coffee_bar']
# X_all_rms['children'] = X_all_rms['total_children'] * X_all_rms['num_children_at_home']
# X_all_rms['children*avg_cars_at home(approx).1'] = X_all_rms['total_children'] * X_all_rms['num_children_at_home'] * X_all_rms['avg_cars_at home(approx).1']
# X_all_rms['store_sqft_encode'] = X_all_rms['store_sqft'].copy()
# X_all_rms['children/avg_cars_at home(approx).1'] = (X_all_rms['children'] / X_all_rms['avg_cars_at home(approx).1']).clip(None, 100)

# add_data['extra_attraction'] = add_data['florist'] + add_data['video_store'] + add_data['prepared_food'] + add_data['coffee_bar']
# add_data['children'] = add_data['total_children'] * add_data['num_children_at_home']
# add_data['children*avg_cars_at home(approx).1'] = add_data['total_children'] * add_data['num_children_at_home']* add_data['avg_cars_at home(approx).1']
# add_data['store_sqft_encode'] = add_data['store_sqft'].copy()
# add_data['children/avg_cars_at home(approx).1'] = (add_data['children'] / add_data['avg_cars_at home(approx).1']).clip(None, 100)
# features_all_rms = X_all_rms.columns.tolist()

# _models_all_rms, _oof_preds_all_rms, mean_train_score_all_rms, mean_valid_score_all_rms, _scalers_all_rms = evaluate_model("XGBRegressor_default", xgb, 
#                                                                 X_all_rms, y_all_rms , use_original=True, original_data=add_data, gbdt=True, n_splits=10, random_state_list=[0, 5, 10]) 